# GitHub API REST v3 -> VSCodeの開発言語情報の取得

## 使い方
- とりあえず一回実行します。
- config\setting.cnfが出来上がります。
    - ユーザー名とパスワード(GitHubのアカウント)をsetting.cnfに記述します。
    - ユーザー名とパスワードは記述しなくても良いですが、1時間にX回しかアクセスできないという制限があります
- リポジトリを指定します
- 実行

## リポジトリの確認と指定方法
例えば以下の条件で調べると, microsoft/vscodeと出てきます。
- microsoft -> owner
- vscode -> repo

https://github.com/search?q=vscode 

In [7]:
# input values
owner = "microsoft"
repo = "vscode"

print("[owner]{0} [repo]{1}について出力します。".format(owner, repo))


[owner]atom [repo]atomについて出力します。


## ライブラリインポート

In [8]:
# Libraries
import requests
import json
import pprint
import pandas as pd

## original Libraries
import config_manager as cnf
import github_api.github_api_manager as gapi

In [9]:
# load config file
cnf_data = cnf.confg_manager()
cnf_data.load()
username, userpassword = cnf_data.get_userparam()

# create github api object
gapi_ctrl = gapi.github_api_manager(username, userpassword)

## リポジトリの基本的な情報を取得します。

In [10]:
# create repository api
github_api = "/repos/{owner}/{repo}".replace("{owner}", owner).replace("{repo}", repo)
ret_json = gapi_ctrl.get_json(github_api)

# select keys
check_list = ["name","full_name","description", "size", "stargazers_count", "watchers_count","forks_count","open_issues_count","network_count","subscribers_count"]


# convert dataframe
d1={}
for k in check_list:
    d1[k]=pd.Series(ret_json[k])

df1=pd.DataFrame(d1)

df1 = df1.T.reset_index().rename(columns={'index': 'key', 0 : 'value'})

# output
print("\n== Repository Information ==\n")
df1.head(20)

GET https://api.github.com/repos/atom/atom

== Repository Information ==



,key,value
0,name,atom
1,full_name,atom/atom
2,description,:atom: The hackable text editor
3,size,319076
4,stargazers_count,52690
5,watchers_count,52690
6,forks_count,14911
7,open_issues_count,879
8,network_count,14911
9,subscribers_count,2525


# リポジトリの言語情報を取得します
色々と便利なのでDataFrameに変換しています。
言語とバイト数が返ってきます

In [11]:
github_api = "/repos/{owner}/{repo}/languages".replace("{owner}", owner).replace("{repo}", repo)
ret_json = gapi_ctrl.get_json(github_api)

d2={}
for k,v in ret_json.items():
    d2[k]=pd.Series(v)

df2=pd.DataFrame(d2)
df2 = df2.T.reset_index().rename(columns={'index': 'language', 0 : 'number of bytes of code'})

# output
print("\n== Repository Languages ==\n")
df2.head(100)

GET https://api.github.com/repos/atom/atom/languages

== Repository Languages ==



,language,number of bytes of code
0,JavaScript,4347824
1,CSS,395100
2,CoffeeScript,244664
3,Shell,7418
4,Batchfile,2431
5,HTML,460
6,Dockerfile,354


In [12]:
df2.to_csv("vscode-languages.csv")